# load the package

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn import metrics

C:\Users\yogit\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\yogit\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
train = pd.read_csv('../../Input Files/Digit Recognizer/train.csv')
test = pd.read_csv('../../Input Files/Digit Recognizer/test.csv')
sample_submission = pd.read_csv('../../Input Files/Digit Recognizer/sample_submission.csv')

In [3]:
col = ['pixel%d' %i for i in range(784)]

# Workflow

## load the dataset

In [4]:
train_backup = train.copy()
test_backup = test.copy()

## train test split

In [5]:
X_train, X_val, Y_train, Y_val = train_test_split(train[col], train['label'], test_size=0.1)

In [10]:
dtrain = xgb.DMatrix(X_train, label=Y_train)
dval = xgb.DMatrix(X_val, label=Y_val)

In [76]:
sample_submission_v2['Label'] = preds

# ver 1 xgboost

In [13]:
watchlist = [(dval,'eval'),(dtrain,'train')]

xgb_params = {
    'objective': 'multi:softmax',
    'eval_metric': 'mlogloss',
    'num_class': 10,
    'max_depth': 12,
    'eta': 0.05,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
}

xgb_clf = xgb.train(xgb_params, dtrain, 2000,
                   watchlist,
                   early_stopping_rounds=20,
                   verbose_eval=20)

[0]	eval-mlogloss:2.10613	train-mlogloss:2.09380
Multiple eval metrics have been passed: 'train-mlogloss' will be used for early stopping.

Will train until train-mlogloss hasn't improved in 20 rounds.
[20]	eval-mlogloss:0.77709	train-mlogloss:0.65947
[40]	eval-mlogloss:0.40789	train-mlogloss:0.26946
[60]	eval-mlogloss:0.25444	train-mlogloss:0.11854
[80]	eval-mlogloss:0.18416	train-mlogloss:0.05633
[100]	eval-mlogloss:0.14936	train-mlogloss:0.02939
[120]	eval-mlogloss:0.13071	train-mlogloss:0.01704
[140]	eval-mlogloss:0.11929	train-mlogloss:0.01086
[160]	eval-mlogloss:0.11196	train-mlogloss:0.00755
[180]	eval-mlogloss:0.10702	train-mlogloss:0.00560
[200]	eval-mlogloss:0.10371	train-mlogloss:0.00439
[220]	eval-mlogloss:0.10116	train-mlogloss:0.00357
[240]	eval-mlogloss:0.09917	train-mlogloss:0.00300
[260]	eval-mlogloss:0.09725	train-mlogloss:0.00258
[280]	eval-mlogloss:0.09589	train-mlogloss:0.00227
[300]	eval-mlogloss:0.09469	train-mlogloss:0.00202
[320]	eval-mlogloss:0.09360	train-mlo

In [15]:
res = xgb_clf.predict(xgb.DMatrix(test[col]))

In [87]:
sample_submission['Label'] = res
sample_submission.to_csv('submission.csv', index=False)

---

# ver 2 xgboost

In [105]:
y = train_backup.label.values
X = train_backup.drop('label', axis=1).values
test = test_backup.copy()
X[X>0] = 1

In [106]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(37800, 784) (4200, 784) (37800,) (4200,)


In [107]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

params = {
    'max_depth': 5,                 # the maximum depth of each tree
    'eta': 0.5,                     # the training step for each iteration
    'eta_decay': 0.9,
    'min_eta': 0.05,
    'silent': 0,                    # logging mode - quiet
    'objective': 'multi:softmax',   # multiclass classification using the softmax objective
    'num_class': 10,                # the number of classes that exist in this datset
    'early_stopping_rounds': 10,
    'n_estimators': 5000
}  

def eval_acc(preds, dtrain):
    labels = dtrain.get_label()
    return 'acc', metrics.accuracy_score(labels, preds)

results = {}
model = xgb.train(params, dtrain, num_boost_round=100, evals=[(dval, 'val')], evals_result=results, feval=eval_acc)

[10:19:16] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { early_stopping_rounds, eta_decay, min_eta, n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	val-merror:0.19024	val-acc:0.80976
[1]	val-merror:0.14000	val-acc:0.86000
[2]	val-merror:0.11690	val-acc:0.88309
[3]	val-merror:0.10881	val-acc:0.89119
[4]	val-merror:0.09738	val-acc:0.90262
[5]	val-merror:0.08857	val-acc:0.91143
[6]	val-merror:0.08119	val-acc:0.91881
[7]	val-merror:0.07809	val-acc:0.92190
[8]	val-merror:0.07190	val-acc:0.92810
[9]	val-merror:0.07048	val-acc:0.92952
[10]	val-merror:0.06548	val-acc:0.93452
[11]	val-merror:0.06143	val-acc:0.93857
[12]	val-merror:0.05810	val-acc:0.94190
[13]	val-merror:0.05524	val-acc:0.94476
[14]	val-

## adding 1/0 to features

In [98]:
y = train.label.values
X = train.drop('label', axis=1).values
test[test>0] = 1
X[X>0] = 1

In [99]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(37800, 784) (4200, 784) (37800,) (4200,)


In [94]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

params = {
    'max_depth': 5,                 # the maximum depth of each tree
    'eta': 0.5,                     # the training step for each iteration
    'eta_decay': 0.9,
    'min_eta': 0.05,
    'silent': 0,                    # logging mode - quiet
    'objective': 'multi:softmax',   # multiclass classification using the softmax objective
    'num_class': 10,                # the number of classes that exist in this datset
    'early_stopping_rounds': 10,
    'n_estimators': 5000
}  

def eval_acc(preds, dtrain):
    labels = dtrain.get_label()
    return 'acc', metrics.accuracy_score(labels, preds)

results = {}
model = xgb.train(params, dtrain, num_boost_round=100, evals=[(dval, 'val')], evals_result=results, feval=eval_acc)

[10:04:32] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { early_stopping_rounds, eta_decay, min_eta, n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	val-merror:0.16976	val-acc:0.83024
[1]	val-merror:0.12738	val-acc:0.87262
[2]	val-merror:0.11500	val-acc:0.88500
[3]	val-merror:0.10024	val-acc:0.89976
[4]	val-merror:0.08929	val-acc:0.91071
[5]	val-merror:0.08310	val-acc:0.91690
[6]	val-merror:0.07881	val-acc:0.92119
[7]	val-merror:0.07214	val-acc:0.92786
[8]	val-merror:0.07024	val-acc:0.92976
[9]	val-merror:0.06405	val-acc:0.93595
[10]	val-merror:0.06095	val-acc:0.93905
[11]	val-merror:0.05857	val-acc:0.94143
[12]	val-merror:0.05381	val-acc:0.94619
[13]	val-merror:0.05214	val-acc:0.94786
[14]	val-

In [100]:
x_test = test.values
dtest = xgb.DMatrix(x_test)

In [101]:
preds = model.predict(dtest)
print(preds.shape)
preds[:2]

(28000,)


array([2., 0.], dtype=float32)

In [102]:
sample_submission_v2.Label = preds.astype(np.int32)

In [103]:
sample_submission_v2.to_csv('submission_v2.csv', index=False)

# XGBOOST ver 3, PCA - t-SNE

In [6]:
y = train_backup.label.values
X = train_backup.drop('label', axis=1).values
test = test_backup.copy()
X[X>0] = 1

from sklearn.manifold import TSNE

In [8]:
from sklearn.decomposition import PCA
pca_161 = PCA(n_components=150)
pca_result_161 = pca_161.fit_transform(X)

In [10]:
pca_tsne = TSNE(random_state = 7, n_components=2, verbose=0, perplexity=30, n_iter=300).fit_transform(pca_result_161)